In [21]:
# Initialize
!git clone --recursive https://github.com/mitosagi/puzzdra-nnsolver
%cd /kaggle/working/puzzdra-nnsolver
!pip install --log=pip_log -e .
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/akashpalrecha/Resnet-multichannel/master/multichannel_resnet.py", "multichannel_resnet.py")

Cloning into 'puzzdra-nnsolver'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 290 (delta 26), reused 0 (delta 0), pack-reused 245
Receiving objects: 100% (290/290), 8.30 MiB | 21.14 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Submodule 'extern/pybind11' (https://github.com/pybind/pybind11) registered for path 'extern/pybind11'
Cloning into '/kaggle/working/puzzdra-nnsolver/puzzdra-nnsolver/puzzdra-nnsolver/extern/pybind11'...
remote: Enumerating objects: 27236, done.        
remote: Counting objects: 100% (8/8), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 27236 (delta 1), reused 4 (delta 1), pack-reused 27228        
Receiving objects: 100% (27236/27236), 10.39 MiB | 21.98 MiB/s, done.
Resolving deltas: 100% (19162/19162), done.
Submodule path 'extern/pybind11': checked out '8de7772cc72daca8e947b79b83fea46214931604'
/kaggle/working/pu

('multichannel_resnet.py', <http.client.HTTPMessage at 0x7e423df1b250>)

In [2]:
import random
import numpy as np
from puzzpy import PuzzTable

drop_color = 3
board_width = 6
board_height = 5

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'

def npUint8(array):
    return np.array(array, dtype=np.uint8)

class PuzzBoard():        
    def reset(self):
        self.prev_action = 255
        
        while True:
            self.table = PuzzTable("".join([str(random.randrange(drop_color)) for i in range(board_width*board_height)]), random.randrange(board_width), random.randrange(board_height), 50) # n色陣　操作時間m秒
            if self.table.eval_otoshi() == 0:
                break

        return npUint8(self.table.get_table())
    
    def step(self):
        next_tables = self.table.next_tables()
        valid_actions = [action for action, table in enumerate(next_tables) if table.get_table()[0][0] != 127 and abs(action - self.prev_action) != 2]
        self.prev_action = random.choice(valid_actions)
        self.table = next_tables[self.prev_action]
        
        return self.prev_action, npUint8([(npUint8(table.get_table()) if action in valid_actions else np.zeros_like(npUint8(table.get_table()))) for action, table in enumerate(next_tables)])
    def render(self):
        tcolor = [bcolors.RED, bcolors.BLUE, bcolors.GREEN, bcolors.MAGENTA, bcolors.YELLOW, bcolors.BLACK]
        start = self.table.get_XY_as_table()
        table = self.table.get_table()
        for i in range(board_height):
            for j in range(board_width):
                if start[i][j] == 1:
                    print(tcolor[table[i][j]-1]  +  bcolors.UNDERLINE + "●" + bcolors.ENDC, end='')
                else:
                    print(tcolor[table[i][j]-1]  + "●" + bcolors.ENDC, end='')
            print('')

In [3]:
env = PuzzBoard()
obs = env.reset()
env.render()

n_steps = 3

for step in range(n_steps):
    print("Step {}".format(step + 1))
    action, obs = env.step()
    with np.printoptions(threshold=np.inf):
        print("action: ", action)
#         print(obs)
        env.render()

●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 1
action:  1
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 2
action:  1
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 3
action:  2
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●


In [17]:
# %%timeit # num_step=50のとき1_000サンプル生成に3.41sかかる。目標の100_000サンプル生成は６分弱。
def make_data(num_step):
    env = PuzzBoard()
    start = env.reset()

    sample_step = random.randrange(1, num_step + 1)

    for i in range(sample_step):
        answer, obs = env.step()
    sample_obs = obs

    for i in range(num_step - sample_step):
        answer, obs = env.step()
    end = obs[answer]
        
    return np.concatenate([npUint8([start]),sample_obs,npUint8([end])]), answer

from torch.utils.data import Dataset

class PuzzDataSet(Dataset):
    def __init__(self):
        samples = [make_data(50) for i in range(10_000)]
        self.data = np.array(([sample[0] for sample in samples]), dtype=np.float32)
        self.labels = np.array(([sample[1] for sample in samples]), dtype=np.float32)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

%time dataset = PuzzDataSet()

CPU times: user 46.6 s, sys: 22.9 ms, total: 46.6 s
Wall time: 46.6 s


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import multichannel_resnet
from multichannel_resnet import get_arch as Resnet
from torchvision.models import resnet50

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_input_channel = 6
        num_classes = 1
        resnet = resnet50()
        resnet.conv1 = nn.Conv2d(num_input_channel, 64, kernel_size=7, stride=2, padding=3,bias=False)
        resnet.fc = nn.Linear(512 * 4, num_classes)
        self.resnet = resnet
    def forward(self, x):
        x = self.resnet(F.interpolate(x, size=None, scale_factor=2, mode='nearest'))
        return x
model = Net()
device = torch.device('cuda')
model = model.to(device)
    
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)


criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=0.002)

import datetime

def train(epoch):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = torch.flatten(output)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))

for epoch in range(1, 10 + 1):
    train(epoch)

[2023-11-03 16:19:43.578997] Train Epoch: 1 [0/10000 (0%)]	Average loss: 0.037717
[2023-11-03 16:19:47.683156] Train Epoch: 2 [0/10000 (0%)]	Average loss: 0.052114
[2023-11-03 16:19:51.680269] Train Epoch: 3 [0/10000 (0%)]	Average loss: 0.030750
[2023-11-03 16:19:55.692179] Train Epoch: 4 [0/10000 (0%)]	Average loss: 0.021683
[2023-11-03 16:19:59.687593] Train Epoch: 5 [0/10000 (0%)]	Average loss: 0.251832
[2023-11-03 16:20:03.678850] Train Epoch: 6 [0/10000 (0%)]	Average loss: 0.013771
[2023-11-03 16:20:07.671964] Train Epoch: 7 [0/10000 (0%)]	Average loss: 0.009678
[2023-11-03 16:20:11.674804] Train Epoch: 8 [0/10000 (0%)]	Average loss: 0.011772
[2023-11-03 16:20:15.689871] Train Epoch: 9 [0/10000 (0%)]	Average loss: 0.010294
[2023-11-03 16:20:19.683694] Train Epoch: 10 [0/10000 (0%)]	Average loss: 0.009193
